# SINGLE INDICATOR THRESHOLD BACKTESTING PSEUDO-EXPERIMENTS

## code performs backtesting using a single indicator with a threshold crossing strategy:

* if indicator crosses below or above buy_threshold -> buy
* if indicator crosses above or below sell_threshold -> sell

All the money available is used for each and every trade starting with $100: money from first trade is injected in full into the second trade etc...

This iteration of the code does not allow to loop over different currencies, or indicators, or timeframes, or indicator timeperiods. Pseudo-experiments are CPU time consumming by nature, therefore focusing on one configuration at a time is recommended. Currently, only TA-lib indicators using one candle information (e.g. 'close' only) are allowed.

Data can be obtained from various CEX via their respective APIs using the CCXT Python module.

There should be data avaible prior to trade start datetime in order for the first trade data point to have meaningful indicator calculation (since indicator requires past data). This is why the code here and there does a 2-day backward time shift.

Data's datetime is the default UTC-0 timezone (France is UTC+1 and NA east cost is UTC-5).

## Various module imports

In [1]:
import itertools
import os
import time
import warnings

from datetime import datetime, timedelta
from random import randrange

import numpy as np
import pandas as pd
import talib as ta

from qtpylib.indicators import crossed
from tqdm import tqdm

from market import CryptoMarket

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.width', 10000)

## Moving to directory containing both data and output folders named respectively: 'data' and 'backtesting_output'

In [2]:
cd D:\crypto\

D:\crypto


In [3]:
os.listdir('.')

['backtesting_output', 'data']

## Various backtesting configurations

In [4]:
# CEX, CURRENCY, FEES, INITIAL TRADING AMOUNT, TIMEFRAME
exchange = 'binance'  # see https://github.com/ccxt/ccxt for available exchanges
trading_pair = 'LINK-USDT'  # see specific exchange (e.g. https://www.binance.com/en/markets) for trading pairs
buy_fee = 0.1  # in percent
sell_fee = 0.1  # in percent
initial_cash = 100  # DO NOT CHANGE or hell will break loose
verbose = 0  # print-out verbosity: 0, 1, 2, 3
timeframe = 5  # timeframe in minute of one candle

# INDICATOR
indicator = 'RSI'  # see TA-lib documentation: https://mrjbq7.github.io/ta-lib/
ind_timeperiod = 28  # number of 'timeframe' periods used to compute the indicator
ind_candle = 'close'  # what to use of candle for indicator: open, high low or close

ind_buy_min = 26  # scan lower buy limit
ind_buy_max = 31  # scan upper buy limit
ind_buy_cross = 'below'  # options are cross 'above' or 'below' the set buy threshold

ind_sell_min = 42  # scan lower sell limit
ind_sell_max = 51  # scan upper buy limit
ind_sell_cross = 'above'  # options are cross 'above' or 'below' the set sell threshold

ind_step_size = 1  # scan step size commong to both buy/sell 


# OUTPUT CSV FILE
csv_output_name = exchange + '_' + trading_pair + '_' + indicator + ind_candle +str(ind_timeperiod) + '_' + str(timeframe) + 'min' + \
                    '_bx-' + ind_buy_cross + '_' + str(ind_buy_min) + '-' + str(ind_buy_max) + \
                    '_sx-' + ind_sell_cross + '_' + str(ind_sell_min) + '-' + str(ind_sell_max) + \
                    '_step' + str(ind_step_size) + '.csv'


# PSEUDO-EXPERIMENT
n_ensemble = 100  # number of pseudo-experiments to run
run_pseudo_experiment = True  # if False, use all the data minus first 2 days, not a random subset
datetime_scan = 'uniform'  # 'uniform' to uniformly sample datetime space, 'more_recent' to sample more recent datetimes


# DATA FETCHING
data_first_datetime = datetime(2019, 7, 1, 0, 0, 0)  # start fetching data from this date (oldest date is exchange dependent)
data_last_datetime = 'now'  # fetch data until data


# TRADE START AND END DATETME, MINIMUM TRADING DAYS
trade_start_datetime = data_first_datetime + timedelta(days=2)  # by default start with earliest available data with 2 days shift
trade_end_datetime = datetime.utcnow()  # UTC-0 (Iceland) is the time zone of the data
minimum_nday_trade = 15  # minimum number of days to trade
nday_trade = trade_end_datetime-trade_start_datetime
nday_trade = nday_trade.days + nday_trade.seconds/3600/24

# Fetch market data

In [5]:
def fetch_data(curr_p, exc, start_date, end_date):
  
    market_history = CryptoMarket(curr_p, exc)
    market_history.get_currency_candle_history(start_date, end_date)
    
fetch_data(trading_pair, exchange, data_first_datetime, data_last_datetime)    

File already exists. Fetching 0 day(s) 3 minute(s) of missing data since last download
2020-12-12 23:01:00 2020-12-12 23:05:00


# Trading function

In [6]:
def trade(df, start_date, stop_date, ind_pair_iterator, n_day):

        df_tmp = pd.DataFrame()
        
        for ind_pair in tqdm (list(ind_pair_iterator), desc="Looping over indicator pairs..."): 
                        
            cash = initial_cash
                
            ind_buy_th = ind_pair[0]
            ind_sell_th = ind_pair[1]
            
            if ind_buy_th >= ind_sell_th: # do not allow buy threshold > sell threshold
                continue
                        
            def trigger(df_copy, indicator):
                df_copy['trigger'] = np.empty(len(df)) * 0
                df_copy.loc[
                    (
                        (df_copy[f'{indicator}'].crossed(float(ind_buy_th), ind_buy_cross))
                        & (df_copy.index > start_date)
                    ),
                    'trigger'] = 1  
                    
                df_copy.loc[
                    (
                        (df_copy[f'{indicator}'].crossed(float(ind_sell_th), ind_sell_cross))
                    ),
                    'trigger'] = -1  

                df_copy = df_copy.loc[df_copy['trigger'] != 0]

                df_copy = df_copy.loc[df_copy['trigger'].shift() != df_copy['trigger']]
                 
                return df_copy
            
            df_copy = trigger(df.copy(), indicator)
            df_copy.dropna(inplace=True) 
           
            if verbose > 2:
                print(df_copy.head(10))            
            
            coin = 0            
            cash_list = []
            profit_list = []
            max_price_drop = []
            delta_t = []
            buy_price = 0
            tot_volume = 0
            n_trans = 0
            left_unsold = 0
            sell_date = datetime(2000, 1, 1, 0, 0, 0)

            bought = False
            sold = False    
                        
            for idx, row in df_copy.iterrows():

                if row['trigger'] == 1 and not bought and not sold:
                    buy_price = row['close']
                    buy_date = idx
                    coin = (1-buy_fee/100)*cash/buy_price
                    tot_volume += cash                     
                    bought = True
                    if verbose > 3 :
                        print('cash', round(cash,2), idx, 'buy price', row['close'], 'ind:', row[indicator])
                    cash = 0
                    continue
                elif  bought and not sold and row['trigger'] == -1:                    
                    sell_price = row['close']
                    cash = sell_price*(1-sell_fee/100)*coin
                    tot_volume += cash
                    coin = 0
                    sold = True
                    sell_date = idx
                    max_price_drop.append((df.loc[(df.index>buy_date) & (df.index<sell_date)]['close'].min()-buy_price)/buy_price*100)    
                    if verbose > 3:
                        print('cash', round(cash,2), idx, 'sell price', sell_price, 'ind:', row[indicator])
                        
                if bought and sold:
                    profit_list.append((sell_price-buy_price)/buy_price*100)
                    bought = False
                    sold = False
                    n_trans += 1
                    dtime = sell_date-buy_date
                    minutes = dtime.seconds/60+dtime.days*(24*60)
                    delta_t.append(minutes)
                    
            if bought and not sold:
                n_trans += 1
                cash = (1-sell_fee/100)*coin*df['close'].iloc[-1]
                dtime = df.index[-1]-buy_date
                sell_date = df.index[-1]
                minutes = dtime.seconds/60+dtime.days/(24*60)
                delta_t.append(minutes)
                left_unsold = 1
                profit_list.append((df['close'].iloc[-1]-buy_price)/buy_price*100)
                if verbose > 3:
                    print('unsold', sell_date, 'sell price', df['close'].iloc[-1])                
                                     
            try:
                new_row = {
                    'trading_pair': trading_pair, 
                    'timeframe': timeframe,
                    'ind_timeperiod': ind_timeperiod,
                    'indicator': indicator,
                    'ind_buy':ind_buy_th, 
                    'ind_sell':ind_sell_th,  
                    'ttime_mean': np.mean(np.array(delta_t))/60,  # from minute to hour
                    'ttime_max': np.max(np.array(delta_t))/60,  # from minute to hour
                    'ttime_min': np.min(np.array(delta_t))/60,  # from minute to hour
                    'ttime_std': np.std(np.array(delta_t))/60,  # from minute to hour
                    'tot_volume':tot_volume,                    
                    'profit_per_day':(cash-initial_cash)/initial_cash*100/n_day,  # in percent per day
                    'profit_ind_mean': np.mean(np.array(profit_list)),                                        
                    'profit_ind_max': np.max(np.array(profit_list)),                          
                    'profit_ind_min': np.min(np.array(profit_list)),
                    'profit_ind_std': np.std(np.array(profit_list)),                    
                    'cash_final':cash,                            
                    'n_trans': n_trans,
                    'max_price_drop': np.min(np.array(max_price_drop)),
                    'n_trans_per_day': n_trans/n_day,                  
                    'start_date': start_date,
                    'stop_date': stop_date,
                    'n_day': n_day,
                    'left_unsold': left_unsold
                    
                }
                if verbose > 1:
                    print(profit_list)
                df_tmp = df_tmp.append(new_row, ignore_index=True)
            except Exception as exc:
                if verbose > 0:
                    print("Exception occured:", exc.args[0])
                continue
                           

        return df_tmp

# Pseudo-experiments loop

In [7]:
df_results = pd.DataFrame()

if ind_candle == 'close':
    df_data_raw = pd.read_csv('data/' + exchange + '_' + trading_pair + '.csv', 
                              index_col='time', 
                              usecols=['time', 'close'])
else:
    df_data_raw = pd.read_csv('data/' + exchange + '_' + trading_pair + '.csv', 
                              index_col='time', 
                              usecols=['time', 'close', ind_candle])

df_data_raw.index = pd.to_datetime(df_data_raw.index)
df_data_raw = df_data_raw.resample('1Min').interpolate()

if ind_candle == 'close':
    resample_logic = {'close': 'last'}
elif ind_candle == 'high':
    resample_logic = {'close': 'last', 'high': 'max'}
elif ind_candle == 'low':
    resample_logic = {'close': 'last', 'low': 'min'}
elif ind_candle == 'open':
    resample_logic = {'close': 'last', 'open': 'first'}
    
sampling_interval = str(timeframe) + 'Min'
df_data_raw = df_data_raw.resample(sampling_interval).apply(resample_logic)

df_data_raw[indicator] = getattr(ta, indicator)(df_data_raw[f'{ind_candle}'], ind_timeperiod)  # work with indicator using only one candle input
df_data_raw.dropna(inplace=True)

ind_buy_list = np.arange(ind_buy_min, ind_buy_max, ind_step_size)
ind_sell_list = np.arange(ind_sell_min, ind_sell_max, ind_step_size)

idx_ens = 0

while idx_ens < n_ensemble:
    
    try:

        # identical iterator created each time to avoid rewinding it (not sure how the rewinding can be done)
        ind_pair_iterator = itertools.product(ind_buy_list, ind_sell_list)
    
        if run_pseudo_experiment:
            # draw random start datetime
            
            if datetime_scan == 'more_recent':
                ens_start_date = trade_start_datetime + \
                                    timedelta(days=randrange(0, int(nday_trade-minimum_nday_trade), 1)) + \
                                    timedelta(hours=randrange(-12, 12, 1)) + \
                                    timedelta(minutes=randrange(-30, 30, 1)) + \
                                    timedelta(seconds=randrange(-30, 30, 1))

                ndays_to_end = trade_end_datetime-ens_start_date
                ndays_to_end = ndays_to_end.days + ndays_to_end.seconds/3600/24
                
                if ndays_to_end <= minimum_nday_trade:
                    continue
                    
                ens_end_date = ens_start_date + timedelta(days=randrange(minimum_nday_trade, int(ndays_to_end), 1)) + \
                                    timedelta(hours=randrange(-12, 12, 1)) + \
                                    timedelta(minutes=randrange(-30, 30, 1)) + \
                                    timedelta(seconds=randrange(-30, 30, 1))   
            
            elif datetime_scan == 'uniform': 
                ens_start_date = trade_start_datetime + \
                                    timedelta(days=randrange(0, int(nday_trade), 1)) + \
                                    timedelta(hours=randrange(-12, 12, 1)) + \
                                    timedelta(minutes=randrange(-30, 30, 1)) + \
                                    timedelta(seconds=randrange(-30, 30, 1))

                ens_end_date = trade_start_datetime + \
                                    timedelta(days=randrange(0, int(nday_trade), 1)) + \
                                    timedelta(hours=randrange(-12, 12, 1)) + \
                                    timedelta(minutes=randrange(-30, 30, 1)) + \
                                    timedelta(seconds=randrange(-30, 30, 1))   
            
            
                ens_nday = ens_end_date - ens_start_date
                ens_nday = ens_nday.days + ens_nday.seconds/3600/24

                if ens_nday < minimum_nday_trade:
                    continue
                      
        else:
            ens_start_date = trade_start_datetime
            ens_end_date = trade_end_datetime 

        ens_nday = ens_end_date - ens_start_date
        ens_nday = ens_nday.days + ens_nday.seconds/3600/24            
        
        idx_ens += 1
        print('Ens:', idx_ens, ', start date:', ens_start_date, ', end date:', ens_end_date)     
        
        time.sleep(0.25)  # to avoid visual glitch between above print-out and tqdm's progress bar

        df_data_trade = df_data_raw.copy()
        df_data_trade = df_data_trade[df_data_trade.index>ens_start_date-timedelta(days=2)]  # -2 day for meaningful indicator
        df_data_trade = df_data_trade[df_data_trade.index<ens_end_date]

        df_trade = trade(df_data_trade, 
                         ens_start_date,
                         ens_end_date,
                         ind_pair_iterator,
                         ens_nday)
        
        # if output CSV file does not exist -> create it with header row
        if os.path.isfile('backtesting_output/' + csv_output_name):
            df_trade.to_csv('backtesting_output/' + csv_output_name, mode='a', index=False, header=False)
        # if output CSV file exists -> append results without header row
        # NB: if the CSV file exist without header row, it will not be added, header row is written only when file is created
        else:
            df_trade.to_csv('backtesting_output/'  + csv_output_name, mode='w', index=False)
            
    except Exception as exc:
        print(exc)
        continue

Ens: 1 , start date: 2019-11-27 06:49:40 , end date: 2020-07-04 09:45:21


Looping over indicator pairs...: 100%|█████████████████████████████████████████████████| 45/45 [00:02<00:00, 17.68it/s]


Ens: 2 , start date: 2019-10-07 21:37:10 , end date: 2020-08-15 02:42:35


Looping over indicator pairs...: 100%|█████████████████████████████████████████████████| 45/45 [00:03<00:00, 12.70it/s]


Ens: 3 , start date: 2020-02-14 04:36:44 , end date: 2020-03-01 03:57:47


...

Looping over indicator pairs...: 100%|█████████████████████████████████████████████████| 45/45 [00:01<00:00, 38.00it/s]


Ens: 99 , start date: 2020-03-28 01:24:37 , end date: 2020-10-31 08:54:28


Looping over indicator pairs...: 100%|█████████████████████████████████████████████████| 45/45 [00:02<00:00, 18.58it/s]


Ens: 100 , start date: 2020-02-10 06:16:27 , end date: 2020-04-15 09:08:15


Looping over indicator pairs...: 100%|█████████████████████████████████████████████████| 45/45 [00:01<00:00, 42.49it/s]
